In [1]:
import numpy as np
import copy
import warnings
from qiskit import (
    Aer,
    QuantumCircuit,
    QuantumRegister,
    ClassicalRegister,
    execute,
    transpile
)
from qiskit.utils.mitigation.fitters import CompleteMeasFitter
from qiskit.ignis.verification.tomography import StateTomographyFitter, state_tomography_circuits
from qiskit.quantum_info import state_fidelity
from scipy.linalg import expm
from numpy.linalg import multi_dot 
from numpy import kron as kr
from sympy import Matrix
import utilities
import os

/var/folders/gq/pyfb45qx3wg52k7kygqcjy840000gn/T/ipykernel_16532/1650351720.py:13: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.verification.tomography import StateTomographyFitter, state_tomography_circuits


In [2]:
dict_permutations = {
    "0000": "0100", 
    "0001": "0000", 
    "0010": "0001", 
    "0011": "0101", 
    "0100": "0010",
    "0101": "0110",
    "0110": "0111",
    "0111": "1011",
    "1000": "0011",
    "1001": "1000",
    "1010": "1001",
    "1011": "1100",
    "1100": "1010", 
}

list_permutations = [4, 0, 1, 5, 2, 6, 7, 11, 3, 8, 9, 12, 10, 13, 14, 15]

B = utilities.permutation_to_matrix(4, list_permutations)
Matrix(B)

Matrix([
[  0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[  0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[  0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[  0,   0,   0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0],
[1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[  0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[  0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[  0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
[  0,   0,   0,   0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0],
[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0],
[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0],
[  0,   0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0],
[  0,  

In [3]:
X = np.array([[0,1],[1,0]]) 
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
Id = np.eye(2)

# Defining the hamiltonian divided in: 
#       - H1: first two qubits interactions.
#       - H2: second two qubits interactions.
H1 = kr(X, kr(X, kr(Id, Id))) + kr(Y, kr(Y, kr(Id, Id))) + kr(Z, kr(Z, kr(Id, Id))) 
H2 = kr(Id, kr(X, kr(X, Id))) + kr(Id, kr(Y, kr(Y, Id))) + kr(Id, kr(Z, kr(Z, Id)))
H3 = kr(Id, kr(Id, kr(X, X))) + kr(Id, kr(Id, kr(Y, Y))) + kr(Id, kr(Id, kr(Z, Z))) 


In [4]:
Matrix(H1)

Matrix([
[1.0,   0,   0,   0,    0,    0,    0,    0,    0,    0,    0,    0,   0,   0,   0,   0],
[  0, 1.0,   0,   0,    0,    0,    0,    0,    0,    0,    0,    0,   0,   0,   0,   0],
[  0,   0, 1.0,   0,    0,    0,    0,    0,    0,    0,    0,    0,   0,   0,   0,   0],
[  0,   0,   0, 1.0,    0,    0,    0,    0,    0,    0,    0,    0,   0,   0,   0,   0],
[  0,   0,   0,   0, -1.0,    0,    0,    0,  2.0,    0,    0,    0,   0,   0,   0,   0],
[  0,   0,   0,   0,    0, -1.0,    0,    0,    0,  2.0,    0,    0,   0,   0,   0,   0],
[  0,   0,   0,   0,    0,    0, -1.0,    0,    0,    0,  2.0,    0,   0,   0,   0,   0],
[  0,   0,   0,   0,    0,    0,    0, -1.0,    0,    0,    0,  2.0,   0,   0,   0,   0],
[  0,   0,   0,   0,  2.0,    0,    0,    0, -1.0,    0,    0,    0,   0,   0,   0,   0],
[  0,   0,   0,   0,    0,  2.0,    0,    0,    0, -1.0,    0,    0,   0,   0,   0,   0],
[  0,   0,   0,   0,    0,    0,  2.0,    0,    0,    0, -1.0,    0,   0,   0,   0,   0],
[

In [5]:
time = np.pi
n_steps = 10

trotter_step = multi_dot([expm(-time/n_steps*H1*1j),expm(-time/n_steps*H2*1j),expm(-time/n_steps*H3*1j)])

Matrix(multi_dot([B, trotter_step, np.linalg.inv(B)])).n(3, chop=True)

Matrix([
[  0.769 - 0.25*I, -0.182 - 0.559*I,                0,                0,               0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,               0],
[ 0.147 - 0.452*I,  0.622 + 0.202*I, -0.182 - 0.559*I,                0,               0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,               0],
[-0.164 - 0.226*I,  0.311 - 0.226*I,  0.622 + 0.202*I, -0.182 - 0.559*I,               0,                0,                0,                0,                0,                0,                0,                0,                0,                0,                0,               0],
[-0.164 + 0.119*I, -0.164 - 0.226*I,  0.147 - 0.452*I,   0.769 - 0.25*I,               0,                0,                0,  